### notebook to perform postprocessing and evaluate how good model actually is at marking start/stops

In [ ]:
# imports

In [ ]:
# load best model pth from checkpoints dir

In [ ]:
# perform inference on test set: currently, only for single sequences of 150

In [ ]:
# look at plot of raw probabilities vs target for a single sequence - make function to plot this

In [ ]:
# apply postprocessing steps, tune hyperparameters:
# gaussian smoothing

In [ ]:
# look at gaussian smoothing plot for a single sequence - use same plotting function for this


In [ ]:
# hysteresis thresholding
# look at hysteresis thresholding plot for a single sequence - use same plotting function for this